# Galton Model

In [1]:
from vpython import *
from math import *
from numpy import arange

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##Realistic model with dimensions of actual Galton board.
##This version uses a constant COR and no friction.

###############
#Functions
#return True if a ball and peg overlap
def checkCollision(sph,cyl):
    pegcenter=cyl.pos+0.5*cyl.axis
    rcc=sph.pos-pegcenter
    dist=mag(rcc)
    if(dist<sph.radius+cyl.radius):
        return True
    else:
        return False

#write a line of data to a file
def writeLineToFile(aList):
    # open file and append data to it
    filename = open("data.txt", "a")
    filename.write('\n')
    filename.write('\t'.join(map(str,aList)))        
    filename.close()
###############

plinko_animation = canvas( width=600, height=400, background=color.white)
plinko_animation.title = 'Galton Board Model'
plinko_animation.caption = 'This version uses a constant COR and no friction.'

###############constants
Ntotal=50 #total number of balls
Rball=0.040/2 #radius of ball in m
Rcyl=0.00635/2 #radius of peg in m
dy=0.0508 #vert distance between adjacent rows
dx=0.0508 #hor distance between adjacent columns
H=1.1176 #height of peg array in m
L=1.575 #length of peg array
T=dy #thickness of walls and floor
y0=H/2+Rball #initial height of ball in m
xshift=0 #move entire board by xshift
rows=H/dy+1 #number of rows
cols=L/dx #number of cols
t=0 #clock
dt=0.001 #time step
m=0.04593 #mass of ball
COR=0.667 #coefficient of restitution in perpendicular dir
CORpar=0.489 #coefficient of rest in parallel dir
#COR=0.46 #coefficient of restitution in perpendicular dir
#CORpar=1 #coefficient of rest in parallel dir
g=10 #gravitational field
maxerror=0.9*dx/2 #max x position of ball when dropped from rest
###############

###############
#create 3D objects
ball=sphere(pos=vector(0,y0,0), radius=Rball, color=color.red)

peglist=[] #list of pegs

i=0 #row number
for rownum in arange(0,rows,1):
    for colnum in arange(0,cols,1):
        p=cylinder(pos=vector(-L/2+0.5*dx*(rownum % 2)+colnum*dx+xshift, H/2-rownum*dy,0.05), axis=vector(0,0,-0.1), radius=Rcyl, color=color.black)
        peglist.append(p)
    
#walls and floor
leftside=box(pos=vector(-L/2-T-dx/2+xshift,0,0), size=vector(T,H,0.1), color=color.green)
rightside=box(pos=vector(L/2+T+dx/2+xshift,0,0), size=vector(T,H,0.1), color=color.green)
floor=box(pos=vector(xshift,-H/2-T/2,0), size=vector(L+dx,T,0.1), color=color.green)
###############

###############
#initial velocity, momentum, force
ball.v=vector(0,0,0)
ball.p=m*ball.v
Fgrav=m*vector(0,-g,0)
###############

###############
#histogram
hgraph=graph(width=650, height=200, title="Counts as a function of position on the floor", xtitle="x", ytitle="N", xmin=-L/2, xmax=L/2 )
hist = gvbars(delta=dx, color=color.blue)

#bins for counting ball when it hits the floor
#center of each bin is at +-dx
#initialize number of counts to zero
#each bin is a list with [xpos,N] where xpos is the center of the bin and N is the number of counts in the bin
bins=[]
#a list of all x positions of bins; frequency of each bin is the number of entries for a particular position
positions=[] 
for x in arange(0,L/2,dx):
#    bins.append([x+dx/2,0])
    bins.append([x+dx,0])

for x in arange(0,-L/2,-dx):
    if(x!=0):
        bins.append([x+dx,0])

#plot zero for all bins in the histogram
for b in bins:
        hist.plot(b)
###############


###############
#randomize initial x position of ball
sign=1
if(random()<0.5): sign=-1
error=random()*sign*maxerror
ball.pos=vector(error,y0,0)
ri=ball.pos
###############


##################
# Data Column Headings
##################
headingList=['ri_x (m)','t (s)', 'vy_ave (m/s)',  'bin_x (m)', 'N']
#writeLineToFile(headingList)
filename = open("data.txt", "w")
filename.write('\t'.join(map(str,headingList)))        
filename.close()
print("heading list: ",headingList)
##################

ballIsRolling=False
thetai=pi/2
vcm=0

n=0 #ball number

while n<Ntotal:
    t=0 #reset clock
#    while ball.pos.y-ball.radius>floor.pos.y+floor.size.y/2:
    while ball.pos.y>floor.pos.y+floor.size.y/2: #top surface of floor is at height of last row of pegs
        rate(100)
        if(not ballIsRolling):
            Fnet=Fgrav
            ball.p=ball.p+Fnet*dt
            ball.pos=ball.pos+ball.p/m*dt
            
            for peg in peglist:
                if(checkCollision(ball,peg)):
                    ball.pos=ball.pos-ball.p/m*dt
                    pegcenter=peg.pos+0.5*peg.axis
                    rcc=ball.pos-pegcenter
                    rp=Rcyl*norm(rcc)
                    ball.v=ball.p/m
                    vperpi=dot(ball.v,norm(rp))*norm(rp)
                    vpari=ball.v-vperpi
                    vperpf=-COR*vperpi
                    vparf=CORpar*vpari
                    vf=vperpf+vparf
                    ball.p=m*vf
                    if(mag(vf)<0.05):
                        ballIsRolling=True
                        thetai=acos(rcc.x/mag(rcc))
                        vcm=mag(vf)
        else:
            rcc=ball.pos-pegcenter
            theta=acos(rcc.x/mag(rcc))
            dv=abs(g*cos(theta)*dt)
            vcm=vcm+dv
#            theta=asin(sin(thetai)-1/2*5/3*vcm*vcm/g/(Rcyl+Rball))
            if(thetai<pi/2):
                theta=asin(sin(thetai)-1/2*5/3*vcm*vcm/g/(Rcyl+Rball))
            else:
                theta=pi-asin(sin(thetai)-1/2*5/3*vcm*vcm/g/(Rcyl+Rball))

            ball.v=vcm*vector(cos(theta),sin(theta),0)
            ball.p=m*ball.v
            ball.pos=pegcenter+(Rball+Rcyl)*vector(cos(theta),sin(theta),0)        
            Fpeg=m*g*sin(theta)-m*vcm*vcm/(Rcyl+Rball)
            
            if(Fpeg<0):
#                print("ball rolled off peg ",theta*180/pi,Fpeg,vcm)
                ballIsRolling=False
#                break
        t=t+dt
        
    for b in bins:
        if(ball.pos.x>b[0]-dx/2 and ball.pos.x<b[0]+dx/2):
            b[1]=b[1]+1
            vdrift=H/t
#            headingList=['ri_x (m)','t (s)', 'vy_ave (m/s)',  'bin_x (m)', 'N']
            dataList=[ri.x,t,vdrift,b[0],b[1]]
            positions.append(b[0])
            print(n," data list: ",dataList)
            writeLineToFile(dataList)
        hist.plot(b)
    
    #reset clock, position, velocity, and momentum of ball
    sign=1
    if(random()<0.5): sign=-1
    error=random()*sign*maxerror
    ball.pos=vector(error,y0,0)    
    ri=ball.pos
    ball.p=vector(0,0,0)
    t=0
    
    #increment ball number
    n=n+1

print(positions)
filename = open("positions.txt", "w")
filename.write(','.join(map(str,positions)))
filename.close()

<IPython.core.display.Javascript object>

5  data list:  [-0.0014287284435027872, 6.0930000000003695, 0.18342360085342724, 0.0, 1]
